# Project 6 - Code Breakers

Nicholas Colan

[ASCII Codes](https://theasciicode.com.ar/)

In [1]:
def adjustKey(key,message):
    kList = list(key)
    mList = list(message)
    if len(key) < len(message): # If the length of key < length of message, repeat key until it is length of message
        for k,m in zip(kList,mList):
            if len(kList) < len(mList):
                kList.append(k)
        key = ''.join(kList)
            
    return key

In [2]:
def stringToASCII(s):
    code_list = []
    for c in s:
        code_list.append(ord(c))
    return code_list

In [3]:
def ASCIItoString(code_list):
    s = ''
    for j in code_list:
        s += chr(j)
    return s

In [4]:
def encrypt(message,key):
    if len(key) < len(message):
        key = adjustKey(key,message) # Key will now be of the same length as the message
    key = stringToASCII(key)
    message = stringToASCII(message) # Both key and message are now lists of corresponding ASCII codes
    
    encryptedMessage = []
    
    for m,k in zip(message,key):
        encryptedMessage.append((k+m)%128)
    return encryptedMessage

In [5]:
msg = 'Top secret!'
key = 'buffalo'

encrypt(msg,key)

[54, 100, 86, 6, 84, 81, 82, 84, 90, 90, 7]

In [6]:
def decrypt(message,key):
    # Message is an encrypted integer list of ASCII codes
    if len(key) < len(message):
        key = adjustKey(key,message) # Key will now be of the same length as the message
    key = stringToASCII(key)
    decryptedMessage = []
    
    for m,k in zip(message,key):
        decryptedMessage.append((m-k)%128)
    return decryptedMessage

In [7]:
ASCIItoString(decrypt(encrypt(msg,key),key))

'Top secret!'

## Getting Words From Online Dictionary & Saving Them to a Local File 
## Getting Encrypted Message and Saving it to a Local File

In [8]:
import requests   
import numpy as np

In [9]:
dictionary = requests.get('https://raw.githubusercontent.com/en-wl/wordlist/master/alt12dicts/5desk.txt')

In [10]:
#with open('dictionary.rtf','w') as dictionaryFile:
#    s = dictionary.text
#    dictionaryFile.write(s)

In [11]:
with open('dictionary.rtf','r') as dFile:
    dictionaryWords = [line.rstrip('\n') for line in dFile]

In [12]:
with open('5desk.txt','r') as encryptedText:
    chars = [line.rstrip() for line in encryptedText]

In [23]:
chars = chars[0].split(' ')
print(chars)

['1']


In [22]:
encryptMsg = [int(i) for i in chars] # Convert string chars into integers and store in list
print(encryptMsg)

[1, 49, 85, 12, 88, 102, 8, 68, 84, 70, 101, 75, 70, 80, 17, 23, 92, 73, 77, 89, 23, 73, 71, 96, 74, 105, 8, 85, 84, 74, 23, 56, 66, 94, 88, 92, 77, 8, 95, 5, 91, 81, 84, 77, 85, 103, 77, 66, 94, 70, 101, 75, 70, 24, 5, 64, 8, 88, 77, 88, 23, 80, 70, 12, 92, 95, 87, 78, 118, 89, 95, 77, 1, 50, 70, 107, 77, 84, 12, 84, 105, 76, 66, 85, 83, 92, 76, 1, 96, 84, 23, 92, 66, 87, 74, 23, 92, 73, 81, 5, 103, 84, 66, 79, 74, 23, 87, 71, 12, 38, 95, 73, 67, 19, 88, 23, 74, 80, 99, 88, 100, 73, 79, 24, 5, 110, 80, 70, 90, 5, 107, 80, 66, 96, 111, 89, 87, 88, 95, 82, 88, 86, 1, 77, 88, 106, 93, 78, 81, 73, 23, 92, 73, 81, 5, 109, 73, 68, 77, 83, 107, 8, 81, 91, 88, 107, 35, 1, 96, 77, 92, 8, 84, 77, 82, 92, 20, 1, 99, 77, 102, 20, 1, 99, 77, 92, 86, 1, 91, 83, 23, 92, 73, 81, 5, 99, 73, 84, 96, 5, 91, 73, 90, 12, 89, 95, 77, 107, 96, 77, 105, 77, 70, 12, 82, 92, 86, 1, 99, 74, 105, 77, 1, 96, 84, 106, 91, 70, 80, 5, 93, 90, 80, 89, 5, 102, 93, 85, 12, 84, 93, 8, 85, 84, 74, 23, 90, 80, 79, 80, 96,

At this point, we have the entire dictionary saved in a list named `dictionaryWords` and the encrypted message saved in a list called `encryptMsg`.

Now we need a way to figure out what word is the key. Need to go thru entire dictionary and test each word -- the key that produces the highest % of dictionary words is probably the correct key.

First, make a function to calculate efficency of the key. In `efficency`, a key's rating will increase the more words it has that also exist in `dictionaryWords`.

During testing for `efficency`, I had extremely long runtimes when iterating through `dictionaryWords` and running `if n in dictionaryWords`, which was a list. After doing some research on runtimes when using the `in` operator, I found that the runtime for `n in [set]` was significantly faster. `n in ___` has an average time complexity of `O(n)` when operating on lists but only `O(1)` average time complexity when operating on a set.

In [15]:
def efficency(dWords,message=str):
    splitMsg = message.split(' ')
    d = set(dWords) # Convert list to a set for faster runtime for `word in dictionary`
    correct = 0
    for word in splitMsg:
        w = word.replace('.','').lower()
        if w in d:
            correct +=1
        elif w.capitalize() in d: # Some entries in the dictionary are capitalized
            correct += 1
    return correct / len(splitMsg)

Now we need to iteratively go through every word in `dictionaryWords` and test each one to see how well it cracks the code. Every word will be inserted along with the encrypted message into `decrypt` and the returned message will be tested using `efficency`.

In [16]:
def findKey(encryptedMsg,possibleKeys):
    potentialKeys = []
    for key in possibleKeys:
        D = decrypt(encryptedMsg,key)
        E = efficency(possibleKeys,ASCIItoString(D))
        potentialKeys.append([E,key])
            
    return potentialKeys

In [17]:
from time import time

In [18]:
start = time()
B = findKey(encryptMsg,dictionaryWords)
print(sorted(B,reverse=True))
print(' ')
print('----------------')
print(' ')
end = time()
print(end-start)

[[0.6787330316742082, 'whale'], [0.5, 'Ruskin'], [0.3333333333333333, 'Ryukyu'], [0.3333333333333333, 'Rudy'], [0.3333333333333333, 'Ruby'], [0.18181818181818182, 'methodologist'], [0.14285714285714285, 'methylmercury'], [0.14285714285714285, 'Rx'], [0.14285714285714285, 'Frankfurter'], [0.13333333333333333, 'Methodistical'], [0.13333333333333333, 'Cambridgeshire'], [0.125, 'subcutaneous'], [0.125, 'carbonaceous'], [0.125, 'Bucharest'], [0.11764705882352941, 'ethicist'], [0.11764705882352941, 'Ayrshire'], [0.11267605633802817, 'shape'], [0.1111111111111111, 'sweats'], [0.1111111111111111, 'nonstrategic'], [0.1111111111111111, 'coquettishness'], [0.10869565217391304, 'Falla'], [0.10526315789473684, 'ethology'], [0.1, 'threnodic'], [0.1, 'lethargically'], [0.1, 'frightening'], [0.1, 'abolitionism'], [0.1, 'Yorkshire'], [0.1, 'Ruth'], [0.1, 'Arkwright'], [0.09523809523809523, 'sapphire'], [0.09523809523809523, 'photosphere'], [0.09523809523809523, 'permanent'], [0.09523809523809523, 'Ther

With how this outputted list is formatted, we can sort the list by the efficency rating of the keys. With reverse sorting, we can get the key that had the highest efficency rating at this list's zeroth index and test it out on the encrypted message.

In [19]:
foundKey = sorted(B,reverse=True)[0][1]
foundKey

'whale'

In [20]:
message = ASCIItoString(decrypt(encryptMsg,foundKey))
message

"\nIt so chanced, that after the Parsee's disappearance, I was he whom\nthe Fates ordained to take the place of Ahab's bowsman, when that\nbowsman assumed the vacant post; the same, who, when on the last day the\nthree men were tossed from out of the rocking boat, was dropped astern.\nSo, floating on the margin of the ensuing scene, and in full sight of\nit, when the halfspent suction of the sunk ship reached me, I was then,\nbut slowly, drawn towards the closing vortex. When I reached it, it had\nsubsided to a creamy pool. Round and round, then, and ever contracting\ntowards the button-like black bubble at the axis of that slowly wheeling\ncircle, like another Ixion I did revolve. Till, gaining that vital\ncentre, the black bubble upward burst; and now, liberated by reason of\nits cunning spring, and, owing to its great buoyancy, rising with great\nforce, the coffin life-buoy shot lengthwise from the sea, fell over, and\nfloated by my side. Buoyed up by that coffin, for almost one who

In [21]:
message = message.replace('\n',' ') # Hide endline characters for readibility purposes
message

" It so chanced, that after the Parsee's disappearance, I was he whom the Fates ordained to take the place of Ahab's bowsman, when that bowsman assumed the vacant post; the same, who, when on the last day the three men were tossed from out of the rocking boat, was dropped astern. So, floating on the margin of the ensuing scene, and in full sight of it, when the halfspent suction of the sunk ship reached me, I was then, but slowly, drawn towards the closing vortex. When I reached it, it had subsided to a creamy pool. Round and round, then, and ever contracting towards the button-like black bubble at the axis of that slowly wheeling circle, like another Ixion I did revolve. Till, gaining that vital centre, the black bubble upward burst; and now, liberated by reason of its cunning spring, and, owing to its great buoyancy, rising with great force, the coffin life-buoy shot lengthwise from the sea, fell over, and floated by my side. Buoyed up by that coffin, for almost one whole day and nig

[Moby Dick Epilogue](https://www.sparknotes.com/lit/mobydick/full-text/epilogue/)